In [9]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import numpy as np

from config import *
from dataset import CustomImageFolder
from model import CNNModel
from utils import *

In [10]:
set_seed(42)
model = CNNModel()
model = load_model(model)

conv_weights = [] 
conv_layers = [] 
for module in model.children():
    if isinstance(module, nn.Conv2d):
        conv_weights.append(module.weight)
        conv_layers.append(module)

Model loaded


/Users/smolling/CNN_RiceVarieties/src/utils.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  # Load the model if the file exists


In [1]:
test_dataset = CustomImageFolder(root=args.test_dir)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)
label_dict = {"breed9": 0, "breed28": 1, "breed41": 2, "other": 3}
label_set = set()
unique_breed = []

for i, (image, label, breed) in enumerate(test_loader):
    breed = breed[0]
    if len(label_set) == 4:
        break
    if label.item() in label_set:
        continue
    print(label.item())
    print(breed)
    label_set.add(label.item())
    unique_breed.append((image, label, breed))

NameError: name 'CustomImageFolder' is not defined

In [ ]:
feature_maps = []  # List to store feature maps
layer_names = []  # List to store layer names
for i, (image, label, breed) in enumerate(unique_breed):
    if i == 2:
        break
    image, label = image.to(device), label.to(device)
    for layer in conv_layers:
        image = layer(image)
        feature_maps.append(image)
        layer_names.append(str(layer))
    processed_feature_maps = []  # List to store processed feature maps
    for feature_map in feature_maps:
        feature_map = feature_map.squeeze(0)  # Remove the batch dimension
        mean_feature_map = torch.sum(feature_map, 0) / feature_map.shape[0]  # Compute mean across channels
        processed_feature_maps.append(mean_feature_map.data.cpu().numpy())
    # Plot the feature maps
    fig = plt.figure(figsize=(30, 50))
    for i in range(len(processed_feature_maps)):
        ax = fig.add_subplot(5, 4, i + 1)
        ax.imshow(processed_feature_maps[i])
        ax.set_title(layer_names[i].split('(')[0], fontsize=30)